In [1]:
"""
Realizar un dashboard usando plotly que muestre algunas de las siguientes características para una o más acciones: 
○ Precios Históricos (Time Series With Range Slider)
○ Balance General 
○ Estado de resultados 
○ Medias móviles 
○ Relative Strength Index (RSI)


Dashboard interactivo creado con Plotly y Dash:
Incluye los precios históricos con un rango deslizante (range slider), medias móviles, y el RSI 
"""

'\nRealizar un dashboard usando plotly que muestre algunas de las siguientes características para una o más acciones: \n○ Precios Históricos (Time Series With Range Slider)\n○ Balance General \n○ Estado de resultados \n○ Medias móviles \n○ Relative Strength Index (RSI)\n\n\nDashboard interactivo creado con Plotly y Dash:\nIncluye los precios históricos con un rango deslizante (range slider), medias móviles, y el RSI \n'

In [2]:
#pip install pandas dash plotly yfinance

In [3]:
import pandas as pd
import numpy as np
import dash
from dash import Dash, html
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

In [14]:
df = pd.read_csv("historical_prices_20241028.csv", parse_dates=["date"])

#cálculo media móvil simple (SMA) basada en los precios de cierre con una ventana de 30 días
def calculate_moving_averages(df, window=30):
    df["MA_" + str(window)] = df["close"].rolling(window=window).mean()

#cálculo el RSI: es una medida que compara la magnitud de las ganancias recientes con las pérdidas recientes (a 14 días)
def calculate_rsi(df, window=14):
    delta = df["close"].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    df["RSI"] = 100 - (100 / (1 + rs))

calculate_moving_averages(df, window=20)
calculate_rsi(df)

#aplicación de Dash
app = dash.Dash(__name__)

#layout del dashboard
#define un dropdown para seleccionar el ticker de la acción, 
#un gráfico de los precios históricos con un rango deslizante
#otro gráfico que muestra el RSI

app.layout = html.Div([
    html.H1("Dashboard de Precios Históricos"),
    
    #dropdown selección ticker
    dcc.Dropdown(
        id="ticker-dropdown",
        options=[{"label": ticker, "value": ticker} for ticker in df["ticker"].unique()],
        value=df["ticker"].unique()[0],
        multi=False
    ),
    
    #gráfica precios históricos con slider
    dcc.Graph(id="historical-prices-graph"),
    
    #gráfica de RSI
    dcc.Graph(id="rsi-graph"),
    
    #slider fechas
    dcc.RangeSlider(
        id="date-range-slider",
        min=0,
        max=len(df["date"])-1,
        value=[0, len(df["date"])-1],
        marks={i: str(date)[:10] for i, date in enumerate(df["date"].dt.date.unique())},
        step=1
    )
])

#callback permite actualizar los gráficos cada vez que cambie el ticker o el rango de fechas seleccionado
@app.callback(
    [Output("historical-prices-graph", "figure"),
     Output("rsi-graph", "figure")],
    [Input("ticker-dropdown", "value"),
     Input("date-range-slider", "value")]
)
def update_graphs(selected_ticker, date_range):
    #filtro ticker seleccionado
    df_filtered = df[df["ticker"] == selected_ticker]
    
    #filtro de fechas
    df_filtered = df_filtered.iloc[date_range[0]:date_range[1]]

    #gráfica de precios históricos
    fig_prices = go.Figure()
    fig_prices.add_trace(go.Scatter(
        x=df_filtered["date"],
        y=df_filtered["close"],
        mode="lines",
        name="Precio de Cierre"
    ))
    fig_prices.add_trace(go.Scatter(
        x=df_filtered["date"],
        y=df_filtered["MA_20"],
        mode="lines",
        name="Media Móvil 30 días",
        line=dict(dash="dash")
    ))
    fig_prices.update_layout(
        title=f'Precios Históricos para {selected_ticker}',
        xaxis_title="Fecha",
        yaxis_title="Precio de Cierre",
        xaxis_rangeslider_visible=True
    )
    
    #gráfica de RSI
    fig_rsi = go.Figure()
    fig_rsi.add_trace(go.Scatter(
        x=df_filtered["date"],
        y=df_filtered["RSI"],
        mode="lines",
        name="RSI"
    ))
    fig_rsi.update_layout(
        title=f"RSI para {selected_ticker}",
        xaxis_title="Fecha",
        yaxis_title="RSI",
        yaxis_range=[0, 100]
    )
    
    return fig_prices, fig_rsi

#aplicación
if __name__ == "__main__":
    app.run_server(debug=True)
